# Secop

## ¿Qué contratos del Estado se van a Cerrar?

Los contratos que requiere ejecutar el estado se publican en la portal de Colombia Compra Eficiente por medio de su plataforma SECOP II y los interesados en participar en la licitación, se registran y entran en el concurso de adjudicación.

Nuestro interés en este proyecto era predecir cuando un contrato adjudicado por el estado tenia la probabilidad más alta de cerrarse según las características de contrato como,  de que Departamento(Estado), el orden, la modalidad, el destino del gasto entre otros aspectos del relevantes de la contratación.

La diferencia entre un contrato cerrado y un contrato terminado es que un contrato cerrado es un contrato que ha sido ejecutado en su totalidad, liquidado y archivado, mientras que un contrato terminado es un contrato que ha sido ejecutado en su totalidad, pero que aún no ha sido liquidado o archivado.

**Un contrato cerrado es un contrato que ha finalizado y ya no está vigente y las partes no pueden realizar ninguna modificación al contrato y no pueden iniciar un proceso de reclamación de daños si alguna de las partes ha incumplido sus obligaciones contractuales**

## Datos

Los datos fuente fueron descargados del portal de Datos Abiertos del Gobierno de Colombia que están disponible a todo el público.

https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-II-Contratos-Electr-nicos/jbjy-vk9h

El archivo original pesa 3.62 Gb por lo tanto se tomo este archivo fue procesado inicialmente para descartar varias columnas de datos que no eran relevante para nuestra proyección y fue realizado en el notebook Filtrado_datos_secop generando un arhivo ideal (datos_filtradosv1.csv) para iniciar con el tratamiento de datos.

**Diccionario de datos**

https://www.datos.gov.co/api/views/jbjy-vk9h/files/839439f9-b3b9-4e53-a28d-ab82e752a1dc?download=true&filename=Diccionario%20de%20Datos%20Abiertos%202022%20Contratos%20Electronicos.pdf

In [1]:
import pandas as pd

### Importando archivo

In [2]:
ruta_archivo = r"../data/raw/SECOP_II_-_Contratos_Electr_nicos.csv"
# cargamos datos originales para generar un archivo filtrado

df = pd.read_csv(ruta_archivo)

column_names = df.columns
print(column_names)

/var/folders/j_/_xz8f5pj26n0_6fzf8ktr7xw0000gn/T/ipykernel_1523/3013530260.py:4: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)


Index(['Nombre Entidad', 'Nit Entidad', 'Departamento', 'Ciudad',
       'Localización', 'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
       'Proceso de Compra', 'ID Contrato', 'Referencia del Contrato',
       'Estado Contrato', 'Codigo de Categoria Principal',
       'Descripcion del Proceso', 'Tipo de Contrato',
       'Modalidad de Contratacion', 'Justificacion Modalidad de Contratacion',
       'Fecha de Firma', 'Fecha de Inicio del Contrato',
       'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
       'Fecha de Fin de Ejecucion', 'Condiciones de Entrega',
       'TipoDocProveedor', 'Documento Proveedor', 'Proveedor Adjudicado',
       'Es Grupo', 'Es Pyme', 'Habilita Pago Adelantado', 'Liquidación',
       'Obligación Ambiental', 'Obligaciones Postconsumo', 'Reversion',
       'Valor del Contrato', 'Valor de pago adelantado', 'Valor Facturado',
       'Valor Pendiente de Pago', 'Valor Pagado', 'Valor Amortizado',
       'Valor Pendiente de Amortizacion', 'Val

In [3]:
# Seleccionamos las columnas que queremos mantener
df = df[['Nombre Entidad', 'Nit Entidad', 'Departamento', 'Ciudad',
         'Localización', 'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
         'Estado Contrato',
         'Descripcion del Proceso', 'Tipo de Contrato',
         'Modalidad de Contratacion', 'Justificacion Modalidad de Contratacion',
         'Fecha de Firma', 'Fecha de Inicio del Contrato',
         'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
         'Fecha de Fin de Ejecucion', 'Condiciones de Entrega',
         'TipoDocProveedor',
         'Es Grupo', 'Es Pyme', 'Habilita Pago Adelantado', 'Liquidación',
         'Obligación Ambiental', 'Obligaciones Postconsumo', 'Reversion',
         'Valor del Contrato', 'Valor de pago adelantado', 'Valor Facturado',
         'Valor Pendiente de Pago', 'Valor Pagado', 'Valor Amortizado',
         'Valor Pendiente de Amortizacion', 'Valor Pendiente de Ejecucion',
         'Saldo CDP',
         'Saldo Vigencia', 'EsPostConflicto', 'Destino Gasto',
         'Origen de los Recursos', 'Dias Adicionados',
         'Nacionalidad Representante Legal',
         'Género Representante Legal',
         'Presupuesto General de la Nacion – PGN',
         'Sistema General de Participaciones', 'Sistema General de Regalías',
         'Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)',
         'Recursos de Credito', 'Recursos Propios',
         'Codigo Entidad', 'Fecha Inicio Liquidacion', 'Fecha Fin Liquidacion',
         'Objeto del Contrato']]

# Guardamos el DataFrame en un archivo CSV
df.to_csv('datos_filtradosv1.csv', index=False)

!mv "datos_filtradosv1.csv" "../data/processed/"

### Cargamos nuevo archivo filtrado

In [4]:
ruta_archivo = r"../data/processed/datos_filtradosv1.csv"
# Cargamos los datos del CSV a un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

/var/folders/j_/_xz8f5pj26n0_6fzf8ktr7xw0000gn/T/ipykernel_1523/493588633.py:3: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)


In [5]:
# Seleccionamos las columnas que queremos mantener
df = df[['Departamento',
         'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
         'Estado Contrato',
         'Tipo de Contrato',
         'Modalidad de Contratacion',
         'Fecha de Firma', 'Fecha de Inicio del Contrato',
         'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
         'Fecha de Fin de Ejecucion',
         'Es Grupo', 'Es Pyme', 'Liquidación',
         'Obligación Ambiental',
         'Valor del Contrato', 'Valor Facturado',
         'Valor Pendiente de Pago', 'Valor Pagado',
         'Valor Pendiente de Ejecucion',
         'Saldo CDP',
         'EsPostConflicto', 'Destino Gasto',
         'Origen de los Recursos', 'Dias Adicionados',
         'Género Representante Legal',
         'Presupuesto General de la Nacion – PGN',
         'Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)',
         'Recursos Propios'
         ]]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716131 entries, 0 to 2716130
Data columns (total 31 columns):
 #   Column                                                              Dtype  
---  ------                                                              -----  
 0   Departamento                                                        object 
 1   Orden                                                               object 
 2   Sector                                                              object 
 3   Rama                                                                object 
 4   Entidad Centralizada                                                object 
 5   Estado Contrato                                                     object 
 6   Tipo de Contrato                                                    object 
 7   Modalidad de Contratacion                                           object 
 8   Fecha de Firma                                                      obje

#### Conversion de las fechas a DateTime

In [6]:
columnas_fecha = ['Fecha de Firma', 'Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato',
                  'Fecha de Inicio de Ejecucion', 'Fecha de Fin de Ejecucion']

# Iterar sobre las columnas y convertirlas al tipo de dato 'date'
for columna in columnas_fecha:
    df[columna] = pd.to_datetime(df[columna], format='%m/%d/%Y', errors='coerce')
    df[columna] = pd.to_datetime(df[columna], format='%Y-%m-%d %H:%M:%S', errors='coerce')

#### Filtrar por el estado del contrato

In [7]:
estados_contrato = ["terminado", "Cerrado", "cedido", "Prorrogado", "Suspendido"]

# Filtrar las filas del dataframe basándote en la columna "Estado Contrato"
df_filtradoNoEjecucion = df[df["Estado Contrato"].isin(estados_contrato)]
df_filtradoNoEjecucion

,Departamento,Orden,Sector,Rama,Entidad Centralizada,Estado Contrato,Tipo de Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,...,Valor Pendiente de Ejecucion,Saldo CDP,EsPostConflicto,Destino Gasto,Origen de los Recursos,Dias Adicionados,Género Representante Legal,Presupuesto General de la Nacion – PGN,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios
0,Santander,Territorial,Servicio Público,Ejecutivo,Centralizada,terminado,Prestación de servicios,Contratación directa,2021-08-17,2021-08-19,...,0.0,13750000,No,Inversión,Distribuido,0,Hombre,0,13750000,0
2,Atlántico,Nacional,Educación Nacional,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2020-02-07,2020-02-07,...,0.0,366080000,No,Inversión,Distribuido,0,No Definido,0,0,0
4,Distrito Capital de Bogotá,Nacional,Hacienda y Crédito Público,Ejecutivo,No Definido,Cerrado,Prestación de servicios,Contratación directa,2019-04-15,2019-02-20,...,1557298.0,66379827,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
9,Distrito Capital de Bogotá,Nacional,agricultura,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2021-01-20,2021-01-21,...,1744200.0,1034967000,No,Inversión,Distribuido,0,Mujer,22032000,0,0
10,Distrito Capital de Bogotá,Nacional,Educación Nacional,Corporación Autónoma,Descentralizada,terminado,Prestación de servicios,Contratación régimen especial,2021-08-05,2021-08-09,...,2640000.0,52800000,No,Funcionamiento,Recursos Propios,0,No Definido,0,0,49500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2716118,Distrito Capital de Bogotá,Territorial,Ley de Justicia,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2022-01-25,2022-02-01,...,7350000.0,24500000,No,Inversión,Recursos Propios,0,No Definido,0,0,24500000
2716120,Cundinamarca,Territorial,No aplica/No pertenece,Ejecutivo,Descentralizada,Cerrado,Prestación de servicios,Contratación directa,2018-08-21,2018-08-21,...,20279040.0,20279040,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
2716124,Huila,Nacional,defensa,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Mínima cuantía,2021-04-06,2021-04-14,...,21000000.0,32550000,No,Funcionamiento,Distribuido,0,No Definido,24500000,0,0
2716128,Distrito Capital de Bogotá,Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,terminado,DecreeLaw092/2017,Contratación régimen especial,2022-03-24,2022-03-29,...,6957491.0,58912135,No,Funcionamiento,Recursos Propios,0,Mujer,0,0,6957491


#### Filtrar por año del contrato

In [8]:
df_filtradoAnio = df_filtradoNoEjecucion[df_filtradoNoEjecucion['Fecha de Inicio del Contrato'].dt.year >= 2019]
df_filtradoAnio

,Departamento,Orden,Sector,Rama,Entidad Centralizada,Estado Contrato,Tipo de Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,...,Valor Pendiente de Ejecucion,Saldo CDP,EsPostConflicto,Destino Gasto,Origen de los Recursos,Dias Adicionados,Género Representante Legal,Presupuesto General de la Nacion – PGN,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios
0,Santander,Territorial,Servicio Público,Ejecutivo,Centralizada,terminado,Prestación de servicios,Contratación directa,2021-08-17,2021-08-19,...,0.0,13750000,No,Inversión,Distribuido,0,Hombre,0,13750000,0
2,Atlántico,Nacional,Educación Nacional,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2020-02-07,2020-02-07,...,0.0,366080000,No,Inversión,Distribuido,0,No Definido,0,0,0
4,Distrito Capital de Bogotá,Nacional,Hacienda y Crédito Público,Ejecutivo,No Definido,Cerrado,Prestación de servicios,Contratación directa,2019-04-15,2019-02-20,...,1557298.0,66379827,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
9,Distrito Capital de Bogotá,Nacional,agricultura,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2021-01-20,2021-01-21,...,1744200.0,1034967000,No,Inversión,Distribuido,0,Mujer,22032000,0,0
10,Distrito Capital de Bogotá,Nacional,Educación Nacional,Corporación Autónoma,Descentralizada,terminado,Prestación de servicios,Contratación régimen especial,2021-08-05,2021-08-09,...,2640000.0,52800000,No,Funcionamiento,Recursos Propios,0,No Definido,0,0,49500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2716115,Atlántico,Nacional,Transporte,Ejecutivo,Descentralizada,Cerrado,Prestación de servicios,Contratación directa,2020-02-02,2020-02-03,...,0.0,5949796000,No,Inversión,Distribuido,0,No Definido,0,0,0
2716118,Distrito Capital de Bogotá,Territorial,Ley de Justicia,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2022-01-25,2022-02-01,...,7350000.0,24500000,No,Inversión,Recursos Propios,0,No Definido,0,0,24500000
2716124,Huila,Nacional,defensa,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Mínima cuantía,2021-04-06,2021-04-14,...,21000000.0,32550000,No,Funcionamiento,Distribuido,0,No Definido,24500000,0,0
2716128,Distrito Capital de Bogotá,Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,terminado,DecreeLaw092/2017,Contratación régimen especial,2022-03-24,2022-03-29,...,6957491.0,58912135,No,Funcionamiento,Recursos Propios,0,Mujer,0,0,6957491


In [9]:
# df_filtradoAnio = df_filtradoAnio.drop_duplicates()
# df_filtradoAnio.info()

In [10]:
df_filtradoAnio.shape

(622333, 31)

#### Generando Nuevas Columnas

In [11]:
df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'
df_filtradoAnio = df_filtradoAnio.drop('Sector', axis=1)

df_filtradoAnio["EsPrestacionServicios"] = df_filtradoAnio['Tipo de Contrato'] == 'Prestación de servicios'
df_filtradoAnio = df_filtradoAnio.drop('Tipo de Contrato', axis=1)

df_filtradoAnio["EsGrupo"] = df_filtradoAnio['Es Grupo'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Grupo', axis=1)

df_filtradoAnio["EsPyme"] = df_filtradoAnio['Es Pyme'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Pyme', axis=1)

df_filtradoAnio["EstaLiquidado"] = df_filtradoAnio['Liquidación'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Liquidación', axis=1)

df_filtradoAnio["EsObligacionAmbiental"] = df_filtradoAnio['Obligación Ambiental'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Obligación Ambiental', axis=1)


df_filtradoAnio["Es PostConflicto"] = df_filtradoAnio['EsPostConflicto'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('EsPostConflicto', axis=1)

df_filtradoAnio["EsRecursosPropios"] = df_filtradoAnio['Origen de los Recursos'] == 'Recursos Propios'
df_filtradoAnio = df_filtradoAnio.drop('Origen de los Recursos', axis=1)


df_filtradoAnio = df_filtradoAnio.drop_duplicates()
df_filtradoAnio.info()

df_filtradoAnio.head(15)

/var/folders/j_/_xz8f5pj26n0_6fzf8ktr7xw0000gn/T/ipykernel_1523/2106044307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'


<class 'pandas.core.frame.DataFrame'>
Index: 549407 entries, 0 to 2716129
Data columns (total 31 columns):
 #   Column                                                              Non-Null Count   Dtype         
---  ------                                                              --------------   -----         
 0   Departamento                                                        549407 non-null  object        
 1   Orden                                                               549407 non-null  object        
 2   Rama                                                                549407 non-null  object        
 3   Entidad Centralizada                                                549407 non-null  object        
 4   Estado Contrato                                                     549407 non-null  object        
 5   Modalidad de Contratacion                                           549407 non-null  object        
 6   Fecha de Firma                                  

,Departamento,Orden,Rama,Entidad Centralizada,Estado Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,...,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios,EsServicioPublico,EsPrestacionServicios,EsGrupo,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,EsRecursosPropios
0,Santander,Territorial,Ejecutivo,Centralizada,terminado,Contratación directa,2021-08-17,2021-08-19,2021-12-24,NaT,...,13750000,0,True,True,False,False,False,False,False,False
2,Atlántico,Nacional,Ejecutivo,Centralizada,cedido,Contratación directa,2020-02-07,2020-02-07,2020-12-31,2020-02-07,...,0,0,False,True,False,False,False,False,False,False
4,Distrito Capital de Bogotá,Nacional,Ejecutivo,No Definido,Cerrado,Contratación directa,2019-04-15,2019-02-20,2019-12-31,NaT,...,0,0,False,True,False,True,False,False,False,False
9,Distrito Capital de Bogotá,Nacional,Ejecutivo,Descentralizada,terminado,Contratación directa,2021-01-20,2021-01-21,2021-08-31,NaT,...,0,0,False,True,False,False,False,False,False,False
10,Distrito Capital de Bogotá,Nacional,Corporación Autónoma,Descentralizada,terminado,Contratación régimen especial,2021-08-05,2021-08-09,2021-12-31,NaT,...,0,49500000,False,True,False,False,False,False,False,True
20,Valle del Cauca,Territorial,Ejecutivo,Centralizada,Cerrado,Contratación directa,2020-11-12,2020-11-17,2020-12-30,NaT,...,3862528,0,False,True,False,False,False,False,False,False
25,Caldas,Territorial,Ejecutivo,Descentralizada,terminado,Contratación directa,2022-12-01,2022-12-01,2022-12-31,NaT,...,3000000,0,True,True,False,False,False,False,False,False
27,Distrito Capital de Bogotá,Territorial,Ejecutivo,Descentralizada,terminado,Contratación régimen especial,2020-02-03,2020-02-01,2020-11-30,2020-02-01,...,0,0,False,True,False,False,False,False,False,False
29,Valle del Cauca,Territorial,Corporación Autónoma,Centralizada,terminado,Contratación directa,2023-03-21,2023-03-21,2023-06-30,NaT,...,20216000,0,False,True,False,False,False,False,False,False
33,Nariño,Nacional,Ejecutivo,Descentralizada,terminado,Contratación régimen especial,2022-09-14,2022-10-15,2022-12-31,NaT,...,0,0,False,False,False,False,False,False,False,False


#### Generando nuevo archivo procedado

In [12]:
df_filtradoAnio.to_csv('datos_filtrados_20230719.csv', index=False)

In [13]:
!ls

Secop.ipynb
datos_filtrados_20230719.csv
secopFiltro2021muestra_profile.html
secopFiltro2021muestra_profile20230718.html
secopFiltro2021muestra_profile_20230719_1923.html


In [14]:
!mv "datos_filtrados_20230719.csv" "../data/processed/"

In [17]:
#muestra_aleatoria = df_filtradoAnio.sample(frac=0.4)
#muestra_aleatoria.shape

In [18]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(muestra_aleatoria, title="Pandas Profiling Report")
# profile.to_file("secopFiltro2021muestra_profile_20230719_1923.html")

#### Separar variable a predecir

In [19]:
# Aplicar one-hot encoding
df_one_hot = pd.get_dummies(df_filtradoAnio['Estado Contrato'])

# Combinar el DataFrame original con las nuevas columnas
df_final = pd.concat([df_filtradoAnio, df_one_hot], axis=1)

# Eliminar la columna original "Estado Contrato" si deseas
df_final = df_final.drop('Estado Contrato', axis=1)

df_final

,Departamento,Orden,Rama,Entidad Centralizada,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,Fecha de Fin de Ejecucion,...,EsGrupo,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,EsRecursosPropios,Cerrado,Suspendido,cedido,terminado
0,Santander,Territorial,Ejecutivo,Centralizada,Contratación directa,2021-08-17,2021-08-19,2021-12-24,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
2,Atlántico,Nacional,Ejecutivo,Centralizada,Contratación directa,2020-02-07,2020-02-07,2020-12-31,2020-02-07,2020-12-31,...,False,False,False,False,False,False,False,False,True,False
4,Distrito Capital de Bogotá,Nacional,Ejecutivo,No Definido,Contratación directa,2019-04-15,2019-02-20,2019-12-31,NaT,NaT,...,False,True,False,False,False,False,True,False,False,False
9,Distrito Capital de Bogotá,Nacional,Ejecutivo,Descentralizada,Contratación directa,2021-01-20,2021-01-21,2021-08-31,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
10,Distrito Capital de Bogotá,Nacional,Corporación Autónoma,Descentralizada,Contratación régimen especial,2021-08-05,2021-08-09,2021-12-31,NaT,NaT,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2716112,Caldas,Territorial,Ejecutivo,Centralizada,Contratación directa,2021-01-28,2021-02-01,2021-12-31,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
2716118,Distrito Capital de Bogotá,Territorial,Ejecutivo,Descentralizada,Contratación directa,2022-01-25,2022-02-01,2023-01-08,NaT,NaT,...,False,False,False,False,False,True,False,False,False,True
2716124,Huila,Nacional,Ejecutivo,Descentralizada,Mínima cuantía,2021-04-06,2021-04-14,2021-11-14,NaT,NaT,...,False,True,False,False,False,False,False,False,False,True
2716128,Distrito Capital de Bogotá,Territorial,Corporación Autónoma,Descentralizada,Contratación régimen especial,2022-03-24,2022-03-29,2022-04-30,NaT,NaT,...,False,False,False,False,False,True,False,False,False,True


In [20]:
estado_contrato = df_final["Cerrado"].copy()
contratos_data = df_final.drop(["terminado","cedido","Suspendido","Cerrado"],axis=1)

#### Split dataset

In [21]:
# Calculate test and validation set size:
original_count = len(df_final)
training_size = 0.60 # 60% of records
test_size = (1 - training_size) / 2


training_count = int(original_count * training_size)
test_count = int(original_count * test_size)
validation_count = original_count - training_count - test_count

print(training_count, test_count, validation_count, original_count)

329644 109881 109882 549407


In [22]:
contratos_data = contratos_data[["Departamento","Orden","Entidad Centralizada","Modalidad de Contratacion","Destino Gasto","Género Representante Legal","EsServicioPublico","EsRecursosPropios","EsGrupo","EsPrestacionServicios","EsPyme","EstaLiquidado","EsObligacionAmbiental","Es PostConflicto","Dias Adicionados","Valor del Contrato","Valor Facturado","Valor Pendiente de Pago","Saldo CDP"]]

In [23]:
contratos_data.head(10)

,Departamento,Orden,Entidad Centralizada,Modalidad de Contratacion,Destino Gasto,Género Representante Legal,EsServicioPublico,EsRecursosPropios,EsGrupo,EsPrestacionServicios,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,Dias Adicionados,Valor del Contrato,Valor Facturado,Valor Pendiente de Pago,Saldo CDP
0,Santander,Territorial,Centralizada,Contratación directa,Inversión,Hombre,True,False,False,True,False,False,False,False,0,13750000,13750000,0,13750000
2,Atlántico,Nacional,Centralizada,Contratación directa,Inversión,No Definido,False,False,False,True,False,False,False,False,0,44789333,44789333,0,366080000
4,Distrito Capital de Bogotá,Nacional,No Definido,Contratación directa,Funcionamiento,No Definido,False,False,False,True,True,False,False,False,0,61707928,60150630,1557298,66379827
9,Distrito Capital de Bogotá,Nacional,Descentralizada,Contratación directa,Inversión,Mujer,False,False,False,True,False,False,False,False,0,22032000,20287800,1744200,1034967000
10,Distrito Capital de Bogotá,Nacional,Descentralizada,Contratación régimen especial,Funcionamiento,No Definido,False,True,False,True,False,False,False,False,0,49500000,46860000,2640000,52800000
20,Valle del Cauca,Territorial,Centralizada,Contratación directa,Inversión,No Definido,False,False,False,True,False,False,False,False,0,3862528,0,3862528,3862528
25,Caldas,Territorial,Descentralizada,Contratación directa,Funcionamiento,Hombre,True,False,False,True,False,False,False,False,0,3000000,3000000,0,3000000
27,Distrito Capital de Bogotá,Territorial,Descentralizada,Contratación régimen especial,Funcionamiento,No Definido,False,False,False,True,False,False,False,False,0,21483864,0,21483864,7884100692
29,Valle del Cauca,Territorial,Centralizada,Contratación directa,Funcionamiento,No Definido,False,False,False,True,False,False,False,False,0,20216000,0,20216000,20216000
33,Nariño,Nacional,Descentralizada,Contratación régimen especial,Funcionamiento,Hombre,False,False,False,False,False,False,False,False,0,5254080,0,5254080,0


In [24]:
from sklearn.model_selection import train_test_split

train_x, rest_x, train_y, rest_y = train_test_split(contratos_data, estado_contrato, train_size=training_count)
test_x, validate_x, test_y, validate_y = train_test_split(rest_x, rest_y, train_size=test_count)

print(len(train_x), len(test_x), len(validate_x))

329644 109881 109882


In [25]:
train_x

,Departamento,Orden,Entidad Centralizada,Modalidad de Contratacion,Destino Gasto,Género Representante Legal,EsServicioPublico,EsRecursosPropios,EsGrupo,EsPrestacionServicios,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,Dias Adicionados,Valor del Contrato,Valor Facturado,Valor Pendiente de Pago,Saldo CDP
2298815,Distrito Capital de Bogotá,Nacional,Centralizada,Mínima cuantía,Funcionamiento,No Definido,False,False,False,False,True,False,False,False,0,74419400,74419400,0,85196417
2012925,Bolívar,Territorial,Descentralizada,Contratación directa,Funcionamiento,No Definido,True,False,False,True,True,False,False,False,0,8000000,8000000,8000000,136100000
160885,Distrito Capital de Bogotá,Nacional,Centralizada,Contratación directa,Funcionamiento,No Definido,False,False,False,True,False,False,False,False,0,23400000,23400000,0,6000000000
891733,Santander,Territorial,Descentralizada,Contratación directa,Inversión,No Definido,False,False,False,True,False,False,False,False,0,60500000,60500000,0,60500000
608176,Meta,Territorial,Centralizada,Contratación directa,Funcionamiento,No Definido,False,False,False,True,True,False,False,False,0,1516667,1516667,0,1866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412323,Distrito Capital de Bogotá,Territorial,Descentralizada,Contratación directa,Inversión,No Definido,False,False,False,True,False,False,False,False,0,39285000,0,39285000,43650000
1981484,Santander,Territorial,Descentralizada,Contratación directa,Inversión,No Definido,False,False,False,True,False,False,False,False,0,4200000,3600000,600000,248433337
2685288,Valle del Cauca,Territorial,Descentralizada,Contratación directa,Funcionamiento,No Definido,False,False,False,True,False,False,False,False,0,22800000,0,22800000,401915000
988567,Huila,Territorial,Centralizada,Mínima cuantía,Funcionamiento,No Definido,True,False,False,False,False,True,False,False,0,9000000,3296835,5703165,9000000


In [26]:
train_x.shape

(329644, 19)

## Codificando Variables - Pipeline

In [27]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import RobustScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

In [28]:
one_hot_encoding = ColumnTransformer([
    (
        'one_hot_encode',
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        [
            "Orden",
            "Entidad Centralizada",
            "Modalidad de Contratacion",
            "Destino Gasto",
            "Género Representante Legal",
            "Departamento",
        ]
    )
])

In [29]:
binarizer = ColumnTransformer([
    (
        'binarizer',
        Binarizer(),
        [
            "EsServicioPublico",
            "EsRecursosPropios",
            "EsGrupo",
            "EsPrestacionServicios",
            "EsPyme",
            "EstaLiquidado",
            "EsObligacionAmbiental",
            "Es PostConflicto",
        ]
    )
])

one_hot_binarized = Pipeline([
    ("binarizer", binarizer),
    ("one_hot_encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
])

In [30]:
scaler = ColumnTransformer([
    ("scaler", RobustScaler(), ["Valor del Contrato","Valor Facturado","Valor Pendiente de Pago","Saldo CDP"])
])

In [31]:
passthrough = ColumnTransformer([
    (
        "passthrough",
        "passthrough",
        [

            "Dias Adicionados",
        ]
    )
])

In [32]:
feature_engineering_pipeline = pipe = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    ("categorical", one_hot_encoding),
                    ("categorical_binarized", one_hot_binarized),
                    ("scaled", scaler),
                    ("pass", passthrough)
                ]
            ),
        )
    ]
)

In [33]:
transformed = feature_engineering_pipeline.fit_transform(train_x)
transformed.shape

(329644, 83)

In [34]:
transformed

array([[ 0.        ,  1.        ,  0.        , ..., -0.1963493 ,
         0.26951375,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.23702015,
         0.51806641,  0.        ],
       [ 0.        ,  1.        ,  0.        , ..., -0.1963493 ,
        29.15039062,  0.        ],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  1.03875363,
         1.81599121,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.11259789,
        -0.10253906,  0.        ],
       [ 0.        ,  0.        ,  1.        , ..., -0.1963493 ,
        -0.12597656,  0.        ]])

## Model Trainning

In [38]:
# Get a fresh copy of the pipeline
from sklearn.base import clone

feature_transformer = clone(feature_engineering_pipeline)

features_train_x = feature_transformer.fit_transform(train_x)
features_validate_x = feature_transformer.transform(validate_x)

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

model = RandomForestClassifier(n_estimators=100)

model.fit(features_train_x, train_y)

RandomForestClassifier()

## Model Validation

In [50]:
from sklearn.metrics import accuracy_score, recall_score

pred_y = model.predict(features_validate_x)

print(accuracy_score(validate_y, pred_y))
print(recall_score(validate_y, pred_y))

0.8496751060228245
0.7857059523531745


## Pipeline Final

In [51]:
final_inference_pipeline = Pipeline([
    ("feature_engineering", clone(feature_engineering_pipeline)),
    ("model", RandomForestClassifier(n_estimators=100))
])

In [52]:
final_training_dataset = pd.concat([train_x, validate_x])
final_training_response = pd.concat([train_y, validate_y])

In [53]:
final_inference_pipeline.fit(final_training_dataset, final_training_response)

Pipeline(steps=[('feature_engineering',
                 Pipeline(steps=[('features',
                                  FeatureUnion(transformer_list=[('categorical',
                                                                  ColumnTransformer(transformers=[('one_hot_encode',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse_output=False),
                                                                                                   ['Orden',
                                                                                                    'Entidad '
                                                                                                    'Centralizada',
                                                                                                    'Modalidad '
                                                                                                    'de '
                                                                                                    'Contratacion',
                                                                                                    'Destino '
                                                                                                    'Gasto',
                                                                                                    'Género '
                                                                                                    'Representante '
                                                                                                    'Legal',
                                                                                                    'Departamento'])]...
                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                 sparse_output=False))])),
                                                                 ('scaled',
                                                                  ColumnTransformer(transformers=[('scaler',
                                                                                                   RobustScaler(),
                                                                                                   ['Valor '
                                                                                                    'del '
                                                                                                    'Contrato',
                                                                                                    'Valor '
                                                                                                    'Facturado',
                                                                                                    'Valor '
                                                                                                    'Pendiente '
                                                                                                    'de '
                                                                                                    'Pago',
                                                                                                    'Saldo '
                                                                                                    'CDP'])])),
                                                                 ('pass',
                                                                  ColumnTransformer(transformers=[('passthrough',
                                                                                                   'passthrough',
                                         

## Model testing

In [54]:
test_pred_y = final_inference_pipeline.predict(test_x)

print(accuracy_score(test_pred_y, test_y))
print(recall_score(test_pred_y, test_y))

0.8543242234781263
0.8256717850287908


In [55]:

test_x[['proba_NoCerrado','proba_Cerrado']] = final_inference_pipeline.predict_proba(test_x)

In [56]:

test_x.head(10)

,Departamento,Orden,Entidad Centralizada,Modalidad de Contratacion,Destino Gasto,Género Representante Legal,EsServicioPublico,EsRecursosPropios,EsGrupo,EsPrestacionServicios,...,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,Dias Adicionados,Valor del Contrato,Valor Facturado,Valor Pendiente de Pago,Saldo CDP,proba_NoCerrado,proba_Cerrado
466059,Santander,Territorial,Descentralizada,Contratación directa,Inversión,Hombre,True,False,False,True,...,False,False,False,0,10800000,10800000,0,10800000,1.000000,0.000000
579005,Casanare,Territorial,Descentralizada,Contratación directa,Inversión,Hombre,False,True,False,True,...,True,False,False,0,11200000,0,11200000,11200000,0.530000,0.470000
2646310,Valle del Cauca,Territorial,Descentralizada,Contratación directa,Inversión,Mujer,False,False,False,True,...,False,False,False,0,12000000,0,12000000,12000000,0.000000,1.000000
969353,Atlántico,Territorial,Descentralizada,Contratación directa,Inversión,Hombre,False,False,False,True,...,False,False,False,0,2700000,2700000,0,1061100000,0.382289,0.617711
1226163,Distrito Capital de Bogotá,Nacional,Descentralizada,Contratación directa,Inversión,No Definido,False,False,False,True,...,False,False,False,0,10951000,0,10951000,11100000,0.890000,0.110000
2316054,Meta,Territorial,Descentralizada,Contratación directa,Funcionamiento,Mujer,False,False,False,True,...,False,False,False,0,20000000,19900000,100000,20000000,0.801000,0.199000
832918,Atlántico,Nacional,Descentralizada,Contratación directa,Inversión,No Definido,False,False,False,True,...,False,False,False,0,11782128,11782128,0,5315120607,0.000000,1.000000
2260838,Norte de Santander,Territorial,Centralizada,Contratación directa,Inversión,Mujer,True,False,False,True,...,False,False,False,0,8000000,8000000,0,8000000,0.168466,0.831534
306833,Distrito Capital de Bogotá,Nacional,Centralizada,Contratación régimen especial,Inversión,Mujer,False,False,False,False,...,False,False,True,0,18070350,17588474,481876,21684421,0.480000,0.520000
2148031,Antioquia,Nacional,Descentralizada,Contratación directa,Funcionamiento,Hombre,False,False,False,True,...,False,False,False,0,27430000,0,27430000,207740000,0.830000,0.170000


In [57]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109881 entries, 466059 to 2375746
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Departamento                109881 non-null  object 
 1   Orden                       109881 non-null  object 
 2   Entidad Centralizada        109881 non-null  object 
 3   Modalidad de Contratacion   109881 non-null  object 
 4   Destino Gasto               109881 non-null  object 
 5   Género Representante Legal  109881 non-null  object 
 6   EsServicioPublico           109881 non-null  bool   
 7   EsRecursosPropios           109881 non-null  bool   
 8   EsGrupo                     109881 non-null  bool   
 9   EsPrestacionServicios       109881 non-null  bool   
 10  EsPyme                      109881 non-null  bool   
 11  EstaLiquidado               109881 non-null  bool   
 12  EsObligacionAmbiental       109881 non-null  bool   
 13  Es PostConfli

## Exportando Modelo

In [58]:
from joblib import dump

dump(final_inference_pipeline, "Secop_pipeline.joblib", compress=5)

['Secop_pipeline.joblib']

In [59]:
#import skops.io as sio

#sio.dump(final_inference_pipeline, "Secop_pipeline.sio")

In [45]:
!mv "Secop_pipeline.joblib" "../models/"
#!mv "Secop_pipeline.sio" "../models/"